In [ ]:
import os
import sys
import importlib
from pathlib import Path
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from itertools import product
from collections import defaultdict

In [2]:
# 获取项目根目录
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

# 导入数据
processed_path = Path(project_root) / 'data' / 'processed'
learning_data = pd.read_csv(processed_path / 'Task2_processed.csv')

In [ ]:
# 导入模型分析
import src.Bayesian_new.utils.plot_utils as plot_utils
importlib.reload(plot_utils)

import src.Bayesian_new.utils.model_evaluation as model_eval
importlib.reload(model_eval)
from src.Bayesian_new.utils.model_evaluation import ModelEval

model_eval = ModelEval()

### 1. accuracy

In [3]:
# 生成每个被试内部的试次编号
learning_data['trial_in_sub'] = learning_data.groupby('iSub').cumcount() + 1

# 计算滑动窗口正确率
learning_data['rolling_accuracy'] = learning_data.groupby('iSub')['feedback'].transform(
    lambda x: x.rolling(16, min_periods=16).mean()
)

In [ ]:
# Define the color palette if not already defined
color_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                 '#9467bd', '#8c564b', '#7f7f7f', '#e377c2']

for cond in sorted(learning_data['condition'].unique()):
    cond_data = learning_data[learning_data['condition'] == cond]
    subjects = cond_data['iSub'].unique()

    if len(subjects) == 0:
        continue

    # 创建子图
    n_cols = 4
    n_subs = len(subjects)
    n_rows = (n_subs + n_cols - 1) // n_cols  # 计算行数
    fig, axs = plt.subplots(n_rows,
                            n_cols,
                            figsize=(25, 5 * n_rows),
                            facecolor='none',
                            squeeze=False)

    # 增加子图之间的间距
    fig.subplots_adjust(hspace=0.5, wspace=0.5)  # 设置纵向和横向间距

    for idx, sub in enumerate(subjects):
        row = idx // n_cols
        col = idx % n_cols
        ax = axs[row, col]

        sub_df = cond_data[cond_data['iSub'] == sub]
        valid_data = sub_df.dropna(subset=['rolling_accuracy'])

        if valid_data.empty:
            ax.text(0.5, 0.5, 'No Valid Data', ha='center', va='center')
            ax.axis('off')
            continue

        # 使用循环颜色
        color = color_palette[idx % len(color_palette)]

        # 绘制每个被试的曲线
        ax.plot(valid_data['trial_in_sub'],
                valid_data['rolling_accuracy'],
                label=f'Subject {sub}',
                color=color,
                linewidth=3,
                alpha=1)

        # 绘制分割线
        max_trial = cond_data['trial_in_sub'].max()
        for x in range(64, max_trial + 1, 64):
            ax.axvline(x=x, color='grey', alpha=0.3, linestyle='dashed', linewidth=1)

        # 设置坐标轴标题和标签
        if col == 0:  # 仅保留最左侧子图的纵坐标标签
            ax.set_yticks([i / 5 for i in range(6)])
            ax.set_yticklabels([f"{i / 5:.1f}" for i in range(6)], fontsize=15)
        else:
            ax.set_yticks([])
            ax.set_ylabel(None)

        if row == n_rows - 1:  # 仅保留最后一行子图的横坐标标签
            if cond == 2:
                ax.set_xticks(range(0, max_trial + 1, 192))
                ax.set_xticklabels(range(0, max_trial + 1, 192), fontsize=15)
            else:
                ax.set_xticks(range(0, max_trial + 1, 64))
                ax.set_xticklabels(range(0, max_trial + 1, 64), fontsize=15)
        else:
            ax.set_xticks([])
            ax.set_xlabel(None)

        # 去掉网格和多余的坐标轴
        ax.grid(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_linewidth(2.0)  # 加粗纵坐标轴线
        ax.spines['bottom'].set_linewidth(2.0)  # 加粗横坐标轴线
        ax.tick_params(width=2.0)  # 加粗刻度线
        
        # 在右下角添加被试编号（按顺序编号）
        ax.text(0.95,
            0.05,
            f"S{idx + 1}",
            transform=ax.transAxes,
            fontsize=30,
            ha='right',
            va='bottom',
            color='black')
            
        # 设置图形内部背景透明
        ax.set_facecolor('none')

        # 为整张图设置横坐标标题
        fig.text(0.5, -0.04, 'Trial', ha='center', fontsize=25)
        # 为整张图设置纵坐标标题
        fig.text(-0.02, 0.5, 'Accuracy', va='center', rotation='vertical', fontsize=25)


    # 隐藏多余的子图
    for idx in range(n_subs, n_rows * n_cols):
        axs.flatten()[idx].axis('off')

    plt.tight_layout()
    plt.show()


### 2. oral

In [ ]:
# 导入模型
from src.Bayesian_new import *

import src.Bayesian_new.problems.model_v1 as model
importlib.reload(model)
from src.Bayesian_new.problems.model_v1 import SingleRationalModel

import src.Bayesian_new.problems.config as config
importlib.reload(config)
from src.Bayesian_new.problems.config import config_base

In [ ]:
limited_hypos_list = {}

for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
    condition = subject_data['condition'].iloc[0]
    model_base = SingleRationalModel(config_base, condition=condition)
    s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
            subject_data["choice"].values,
            subject_data["feedback"].values)

    # 口头汇报限制假设集
    data_1 = (subject_data[["feature1_oral", "feature2_oral", "feature3_oral", "feature4_oral"]].values,
            subject_data["choice"].values)
    limited_hypos_list[iSub] = model_base.oral_generate_hypos(data_1)

In [ ]:
oral_hypo_list = {}

for iSub, hypos in limited_hypos_list.items():
    condition = learning_data[learning_data['iSub'] ==
                              iSub]['condition'].iloc[0]
    target_value = 0 if condition == 1 else 42

    hits = []  # 用于存储每个 trial 的 hit 值
    for trial_hypos in hypos:
        hit = 1 if trial_hypos[0] == target_value else 0
        hits.append(hit)  # 将 hit 值添加到列表中

    # 计算滑动平均
    window_size = 16
    hits_avg = pd.Series(hits).rolling(window=window_size, min_periods=window_size).mean().tolist()

    oral_hypo_list[iSub] = {'iSub': iSub, 'condition': condition, 'hit': hits_avg}

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures_sub'
model_eval.plot_oral_hypo_list(oral_hypo_list, plot_path/'M_base', 19)

### 3. model

In [ ]:
# 加载拟合结果
result_path = Path(project_root) / 'results' / 'Bayesian_new'
base_results = joblib.load(result_path / 'M_Base_fitting_results.joblib')
forget_results = joblib.load(result_path / 'M_Fgt_fitting_results_400.joblib')
c1_results = joblib.load(result_path / 'M_Fgt_c1_fitting_results.joblib')

In [ ]:
import src.Bayesian_new.problems.model as model
importlib.reload(model)
from src.Bayesian_new.problems.model import SingleRationalModel

import src.Bayesian_new.problems.config as config
importlib.reload(config)
from src.Bayesian_new.problems.config import config_base

In [ ]:
# 对比模型预测和实际数据
base_predict_results = {}

for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
    condition = subject_data['condition'].iloc[0]
    model_base = SingleRationalModel(config_base, condition=condition)
    s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
            subject_data["choice"].values,
            subject_data["feedback"].values, 
            subject_data["category"].values)

    sub_results = base_results[iSub]
    step_results = sub_results['step_results']
    results = model_base.predict_choice(s_data, step_results, use_cached_dist=False, window_size=16)

    base_predict_results[iSub] = {
        'condition': condition,
        'true_acc': results['true_acc'],
        'pred_acc': results['pred_acc'],
        'sliding_true_acc': results['sliding_true_acc'],
        'sliding_pred_acc': results['sliding_pred_acc'],
        'sliding_pred_acc_std': results['sliding_pred_acc_std']
    }

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures'
# 绘制最优参数变化图
model_eval.plot_params_over_trials(base_results, 'best_beta', plot_path/'M_base')

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(base_results, False, plot_path/'M_base')

# 绘制模型预测和实际数据对比图
model_eval.plot_accuracy_comparison(base_results, plot_path / 'M_base')

# 绘制累积后验概率图
model_eval.plot_hypo_posterior_sums(base_results, limited_hypos_list, plot_path/'M_base')

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures_sub'
# 绘制最优参数变化图
model_eval.plot_params_over_trials(base_results, 'best_beta', plot_path/'M_base', 5)

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(base_results, False, plot_path/'M_base', 5)

# 绘制模型预测和实际数据对比图
model_eval.plot_accuracy_comparison(base_predict_results, plot_path/'M_base', 10)

# 绘制累积后验概率图
model_eval.plot_hypo_posterior_sums(base_results, limited_hypos_list, plot_path/'M_base', 12)

In [ ]:
import src.Bayesian_new.problems.forget as forget
importlib.reload(forget)
from src.Bayesian_new.problems.forget import ForgetModel

import src.Bayesian_new.problems.config as config
importlib.reload(config)
from src.Bayesian_new.problems.config import config_fgt

In [ ]:
# 对比模型预测和实际数据
forget_predict_results = {}

for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
    condition = subject_data['condition'].iloc[0]
    model_fgt = ForgetModel(config_fgt, condition=condition)
    s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
            subject_data["choice"].values,
            subject_data["feedback"].values, 
            subject_data["category"].values)

    sub_results = forget_results[iSub]
    best_step_results = sub_results['step_results']
    results = model_fgt.predict_choice(s_data, best_step_results, use_cached_dist=False, window_size=16)

    forget_predict_results[iSub] = {
        'condition': condition,
        'true_acc': results['true_acc'],
        'pred_acc': results['pred_acc'],
        'sliding_true_acc': results['sliding_true_acc'],
        'sliding_pred_acc': results['sliding_pred_acc'],
        'sliding_pred_acc_std': results['sliding_pred_acc_std']
    }

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures'
# 绘制最优参数变化图
model_eval.plot_params_over_trials(forget_results, 'best_beta', plot_path/'M_fgt')

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(forget_results, False, plot_path/'M_fgt')

# 绘制模型预测和实际数据对比图
model_eval.plot_accuracy_comparison(forget_predict_results, plot_path / 'M_fgt')

# 绘制error grid图
model_eval.plot_error_grids(forget_results, plot_path / 'M_fgt')

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures_sub'
# 绘制最优参数变化图
model_eval.plot_params_over_trials(forget_results, 'best_beta', plot_path/'M_fgt', 5)

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(forget_results, False, plot_path/'M_fgt', 5)

# 绘制模型预测和实际数据对比图
model_eval.plot_accuracy_comparison(forget_predict_results, plot_path/'M_fgt', 10)

In [ ]:
# 对比模型预测和实际数据
c1_predict_results = {}

for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
    # if iSub != 11:
    #     continue
    condition = subject_data['condition'].iloc[0]
    model_fgt = ForgetModel(config_fgt, condition=condition)
    s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
            subject_data["choice"].values,
            subject_data["feedback"].values,
            subject_data["category"].values)

    sub_results = c1_results[iSub]
    best_step_results = sub_results['step_results']
    results = model_fgt.predict_choice(s_data, best_step_results, use_cached_dist=False, window_size=16)

    c1_predict_results[iSub] = {
        'condition': condition,
        'true_acc': results['true_acc'],
        'pred_acc': results['pred_acc'],
        'sliding_true_acc': results['sliding_true_acc'],
        'sliding_pred_acc': results['sliding_pred_acc'],
        'sliding_pred_acc_std': results['sliding_pred_acc_std']
    }

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures'
# 绘制最优参数变化图
model_eval.plot_params_over_trials(c1_results, 'best_beta', plot_path/'M_fgt_c1')

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(c1_results, True, plot_path / 'M_fgt_c1')

# 绘制模型预测和实际数据对比图
model_eval.plot_accuracy_comparison(c1_predict_results, plot_path / 'M_fgt_c1')

# 绘制error grid图
model_eval.plot_error_grids(c1_results, plot_path/'M_fgt_c1')

In [ ]:
plot_path = Path(project_root) / 'results' / 'Figures_sub'
# 绘制最优参数变化图
model_eval.plot_params_over_trials(c1_results, 'best_beta', plot_path/'M_fgt_c1', 5)

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(c1_results, True, plot_path/'M_fgt_c1', 10)

# 绘制模型预测和实际数据对比图
model_eval.plot_accuracy_comparison(c1_predict_results, plot_path/'M_fgt_c1', 10)

In [ ]:
# 绘制累积后验概率图
model_eval.plot_hypo_posterior_sums(base_results, forget_results, c1_results, limited_hypos_list, plot_path/'M_base_fgt')